<a href="https://colab.research.google.com/github/leegonzales/LangchainExamples/blob/main/LangChain_Agent_Updates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up Environment

Install packages and set environment variables

In [ ]:
!pip install openai faiss-cpu google-search-results
!pip install langchain==0.0.58

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPAPI_API_KEY"] = ""

# Tool Priority
An example of giving a tool more/less priority than other ones.

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Music Search",
        func=lambda x: "'All I Want For Christmas Is You' by Mariah Carey.", #Mock Function
        description="A Music search engine. Use this more than the normal search if the question is about Music, like 'who is the singer of yesterday?' or 'what is the most popular song in 2022?'",
    )
]

agent = initialize_agent(tools, OpenAI(temperature=0), agent="zero-shot-react-description", verbose=True)

In [ ]:
agent.run("what is the most famous song of christmas")



> Entering new AgentExecutor chain...
 I should use a music search engine to find the answer
Action: Music Search
Action Input: most famous song of christmas
Observation: 'All I Want For Christmas Is You' by Mariah Carey.
Thought: I now know the final answer
Final Answer: 'All I Want For Christmas Is You' by Mariah Carey.

> Finished chain.


"'All I Want For Christmas Is You' by Mariah Carey."

# Returning Directly
An example of returning the output of a tool directly.

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

In [ ]:
llm_math_chain = LLMMathChain(llm=OpenAI(temperature=0))
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
        return_direct=True
    )
]

In [ ]:
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [ ]:
agent.run("whats 2**.12")



> Entering new AgentExecutor chain...
 I need to calculate this
Action: Calculator
Action Input: 2**.12
Observation: Answer: 1.2599210498948732


> Finished chain.


'Answer: 1.2599210498948732'

# Conversation Agent
Using an agent optimized for conversation.

In [ ]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

In [ ]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
]

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")


In [ ]:
llm=OpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent="conversational-react-description", verbose=True, memory=memory)


In [ ]:
agent_chain.run(input="hi, i am bob")




> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Hi Bob, nice to meet you! How can I help you today?

> Finished chain.


'Hi Bob, nice to meet you! How can I help you today?'

In [ ]:
agent_chain.run(input="what's my name?")




> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Your name is Bob!

> Finished chain.


'Your name is Bob!'

In [ ]:
agent_chain.run("what are some good dinners to make this week, if i like thai food?")




> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: If you like Thai food, some great dinner options this week could include Thai green curry, Pad Thai, or a Thai-style stir-fry. You could also try making a Thai-style soup or salad. Enjoy!

> Finished chain.


'If you like Thai food, some great dinner options this week could include Thai green curry, Pad Thai, or a Thai-style stir-fry. You could also try making a Thai-style soup or salad. Enjoy!'

In [ ]:
agent_chain.run(input="whats the current temperature in pomfret?")




> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: Temperature in Pomfret
Observation: It is 45 degrees fahrenheit, or 7 degrees celsius and feels like 44 degrees fahrenheit.
Thought: Do I need to use a tool? No
AI: The current temperature in Pomfret is 45 degrees Fahrenheit, or 7 degrees Celsius, and it feels like 44 degrees Fahrenheit.

> Finished chain.


'The current temperature in Pomfret is 45 degrees Fahrenheit, or 7 degrees Celsius, and it feels like 44 degrees Fahrenheit.'